In [18]:
import json

from pathlib import Path

from kptncook.models import Recipe
from kptncook.mealie import kptncook_to_mealie

In [17]:
example_path = Path.cwd().parent / "tests" / "fixtures" / "kptncook_example.json"
with example_path.open("r") as f:
    example = json.load(f)
recipe = Recipe.parse_obj(example)

In [20]:
mealie_recipe = kptncook_to_mealie(recipe)

In [21]:
mealie_recipe

Recipe(id=None, user_id=None, group_id=None, name='Chili sin Carne', slug='', image=None, recipe_yield=None, total_time=None, prep_time='30', cook_time=None, perform_time=None, description='', recipe_category=[], tags=[], tools=[], rating=None, org_url=None, recipe_ingredient=[RecipeIngredient(title='Koriander, frisch', note=None, unit=None, food=None, disable_amount=True, quantity=1), RecipeIngredient(title='Tomatenmark', note=None, unit=None, food=None, disable_amount=True, quantity=1), RecipeIngredient(title='Mais, Dose od. Glas', note=None, unit=None, food=None, disable_amount=True, quantity=1), RecipeIngredient(title='Saure Sahne', note=None, unit=None, food=None, disable_amount=True, quantity=1), RecipeIngredient(title='Avocado', note=None, unit=None, food=None, disable_amount=True, quantity=1), RecipeIngredient(title='Speisezwiebel', note=None, unit=None, food=None, disable_amount=True, quantity=1), RecipeIngredient(title='Limette', note=None, unit=None, food=None, disable_amoun